<a href="https://colab.research.google.com/github/tthuy123/graduation/blob/main/%5B4_2%5D_CNN%2BLSTM_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

so this one code is re-implementation/adopted from this paper, i techinacally first follow their method to see if the result works. After that i will see hehehehehe


## import data to use

In [2]:
import pandas as pd

In [4]:
df_student_info = pd.read_csv("/content/drive/MyDrive/OULAD/studentInfo.csv")
df_student_vle = pd.read_csv("/content/drive/MyDrive/OULAD/studentVle.csv")
df_student_assessments = pd.read_csv("/content/drive/MyDrive/OULAD/studentAssessment.csv")
df_assessments = pd.read_csv("/content/drive/MyDrive/OULAD/assessments.csv")
df_courses = pd.read_csv("/content/drive/MyDrive/OULAD/courses.csv")
df_regis = pd.read_csv("/content/drive/MyDrive/OULAD/studentRegistration.csv")
df_vle = pd.read_csv("/content/drive/MyDrive/OULAD/vle.csv")

## check data

In [ ]:
df_student_info.head(5)

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


In [ ]:
df_vle.head(5)

,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1


In [ ]:
df_assessments.head(5)

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [ ]:
display(df_assessments['assessment_type'].unique())

array(['TMA', 'Exam', 'CMA'], dtype=object)

In [ ]:
df_student_assessments.head(5)

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [ ]:
df_courses.head(5)

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [ ]:
df_regis.head(5)

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


In [ ]:
df_vle.head(5)

,id_site,code_module,code_presentation,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,NaN,NaN
1,546712,AAA,2013J,oucontent,NaN,NaN
2,546998,AAA,2013J,resource,NaN,NaN
3,546888,AAA,2013J,url,NaN,NaN
4,547035,AAA,2013J,resource,NaN,NaN


In [ ]:
nan_percentage_week_from = df_vle['week_from'].isnull().sum() / len(df_vle) * 100
nan_percentage_week_to = df_vle['week_to'].isnull().sum() / len(df_vle) * 100

print(f"Percentage of NaN in 'week_from': {nan_percentage_week_from:.2f}%")
print(f"Percentage of NaN in 'week_to': {nan_percentage_week_to:.2f}%")

Percentage of NaN in 'week_from': 82.39%
Percentage of NaN in 'week_to': 82.39%


## import library

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE

## data preprocessing process 1

In [ ]:
## Merge data and get total click for one student
# vle_agg = df_vle.groupby(['id_student', 'code_module', 'code_presentation'])['sum_click'].sum().reset_index()
# vle_agg.rename(columns={'sum_click': 'total_clicks'}, inplace=True)
# vle_agg


### target encoding
Hiện tại đang triển khai bài toán binary classification
WF-PD task: xếp "Distinction" và "Pass" là một nhóm với nhãn Pass và "Fail" cùng "Withdrawn" là nhãn còn lại.

In [6]:
# 1. target encoding
df_student_info['target'] = df_student_info['final_result'].apply(lambda x: 1 if x in ['Distinction', 'Pass'] else 0) # target encoding
## df_info

### các bước xử lý dữ liệu cơ bản, hiện tại dữ liệu tương tác đang được gộp tất cả theo tuần

In [9]:
# 2. xử lý dữ liệu vle clicks (dưới dạng time-series)
# goal: tạo chuỗi lịch sử tương tác của học sinh theo các tuần
MAX_WEEKS = 40 # Chọn 40 vì dựa theo bảng courses max length là 269 ngày xấp xỉ 38.4 tuần
def map_date_to_week(date_val):
  if date_val < 0:
    return 0
  w = date_val // 7

  return w if w < MAX_WEEKS else MAX_WEEKS -1
df_student_vle['week_idx'] = df_student_vle['date'].apply(map_date_to_week)

week_activity = df_student_vle.groupby(['id_student', 'code_module', 'code_presentation', 'week_idx'])['sum_click'].sum().reset_index()

pivot_data = week_activity.pivot_table(
    index=['id_student', 'code_module', 'code_presentation'],
    columns='week_idx',
    values='sum_click',
    fill_value=0)
expected_cols = list(range(MAX_WEEKS)) # [0, 1, ..., 39]
pivot_data = pivot_data.reindex(columns=expected_cols, fill_value=0)

pivot_data.columns = [f'week_{i}' for i in pivot_data.columns]
pivot_data.reset_index(inplace=True)
# pivot_data


Nhận xét về CMA (Computer marked assessment) thấy rằng phần nhiều những bài assessment này có trọng số điểm bằng 0.
Mong muốn khai thác dữ liệu điểm số này theo hai hướng time-series chạy song song với chuỗi click ở phía trên.
1. Chuỗi điểm số: Tính điểm sinh viên đạt được trong 1 tuần
2. Chuỗi độ trễ: Sinh viên nộp sớm hay muộn bao nhiêu ngày.

Như vậy cấu trúc dữ liệu đầu vào mô hình Thay vì (Samples, 40, 1) chỉ có click, giờ ta sẽ có (Samples, 40, 3):
Channel 1: Số lượng Click (Activity)

Channel 2: Điểm số trung bình tuần đó (Performance)

Channel 3: Số ngày nộp sớm/muộn (Discipline)

In [10]:
# 3. XỬ LÝ DỮ LIỆU ASSESSMENT (ĐIỂM SỐ)
assess_full = pd.merge(df_student_assessments, df_assessments, on='id_assessment', how='inner')

# 1. Xử lý score
assess_full['score'] = assess_full['score'].fillna(0) # score không có thì coi là bằng 0

# 2. Tính độ trễ
# Công thức: Ngày nộp - Deadline
# Giá trị dương (+): Nộp muộn (Ví dụ: nộp ngày 30, deadline 25 -> +5)
# Giá trị âm (-): Nộp sớm (Ví dụ: nộp ngày 20, deadline 25 -> -5)
# Lưu ý: Cần xử lý NaN ở cột 'date' (deadline) nếu có.
assess_full['lateness'] = assess_full['date_submitted'] - assess_full['date']
assess_full['lateness'] = assess_full['lateness'].fillna(0) # Nếu không có deadline thì coi như đúng hạn

# 3. Map sang tuần (week index) dựa trên date_submitted
def map_to_week(day):
    if pd.isna(day) or day < 0: return 0
    w = int(day // 7)
    return min(w, 39) # Kẹp trần ở tuần 39

assess_full['week_idx'] = assess_full['date_submitted'].apply(map_to_week)

# 2. Tạo pivot table
# 2.1 Pivot bảng điểm (Score Series)
# Nếu 1 tuần nộp nhiều bài -> Tính trung bình điểm tuần đó
score_pivot = assess_full.pivot_table(
    index=['id_student', 'code_module', 'code_presentation'],
    columns='week_idx',
    values='score',
    aggfunc='mean',
    fill_value=0
    # QUAN TRỌNG: Fill -1 (hoặc 0) cho những tuần KHÔNG có bài kiểm tra.
    # chọn -1 để mô hình phân biệt được "được 0 điểm" và "không có bài".
    # Tuy nhiên, nếu dùng ReLU activation sau này thì nên để 0. Ta tạm để 0 cho an toàn với CNN.
).fillna(0)

# 2.2 Pivot bảng độ trễ (Lateness Series)
lateness_pivot = assess_full.pivot_table(
    index=['id_student', 'code_module', 'code_presentation'],
    columns='week_idx',
    values='lateness',
    aggfunc='mean', # Trung bình độ trễ các bài trong tuần
    fill_value=0 # Không nộp bài tuần đó -> coi như không trễ (0)
).fillna(0)

# 2.3 Pivot Weight (Trọng số) - QUAN TRỌNG
# Đây là kênh thông tin mới: Cho biết tuần đó có bài quan trọng không?
weight_pivot = assess_full.pivot_table(
    index=['id_student', 'code_module', 'code_presentation'],
    columns='week_idx',
    values='weight',
    aggfunc='max', # Nếu tuần đó có nhiều bài, lấy bài có trọng số cao nhất
    fill_value=0
)

# đồng bộ hóa
# Đảm bảo đủ 40 tuần (từ week_0 đến week_39) giống bảng Click
expected_cols = list(range(40))

# Hàm reindex chuẩn hóa
def align_columns(df, prefix):
    df = df.reindex(columns=expected_cols, fill_value=0)
    df.columns = [f'{prefix}_{i}' for i in df.columns]
    return df.reset_index()

df_score_ts = align_columns(score_pivot, 'score_wk')
df_lateness_ts = align_columns(lateness_pivot, 'late_wk')
df_weight_ts = align_columns(weight_pivot, 'weight_wk') # Chuỗi thời gian của trọng số




In [ ]:
df_score_ts

In [ ]:
df_lateness_ts

In [11]:
# 4. Gộp dữ liệu
df_merged = df_student_info.copy()

# merge với dữ liệu click
df_merged = pd.merge(df_merged, pivot_data, on=['id_student', 'code_module', 'code_presentation'], how='left')
# merge với dữ liệu điểm số
df_merged = pd.merge(df_merged, df_score_ts, on=['id_student', 'code_module', 'code_presentation'], how='left')
# merge với dữ liệu độ trễ
df_merged = pd.merge(df_merged, df_lateness_ts, on=['id_student', 'code_module', 'code_presentation'], how='left')

df_merged = pd.merge(df_merged, df_weight_ts, on=['id_student', 'code_module', 'code_presentation'], how='left')

# Fill NaN bằng 0

df_merged.fillna(0, inplace=True) # Fill NaN bằng 0

print(f"Kích thước sau khi merge: {df_merged.shape}")

Kích thước sau khi merge: (32593, 173)


### Xử lý dữ liệu để sẵn sàng đưa vào train
hiện tại đã có df_merged

In [12]:
# Feature Engineering

# Xác định các nhóm cột
ts_click_cols = [f'week_{i}' for i in range(40)]
ts_score_cols = [f'score_wk_{i}' for i in range(40)]
ts_late_cols  = [f'late_wk_{i}' for i in range(40)]
ts_weight_cols = [f'weight_wk_{i}' for i in range(40)]

all_ts_cols = ts_click_cols + ts_score_cols + ts_late_cols + ts_weight_cols

# Xác định các cột static
exclude_cols = ['id_student', 'code_module', 'code_presentation', 'final_result', 'target', 'date_registration', 'date_unregistration', 'region'] + all_ts_cols
static_cols = [c for c in df_merged.columns if c not in exclude_cols]
print(f"Các cột Static được sử dụng: {static_cols}")

# Encode static sử dụng one-hot encoder -> không sử dụng one-hot encoder nữa, vừa tạo nhiều dữ liệu mà có thể
# không mang nhiều ý nghĩa cho dự đoán của model
# Với các biến có thử tự sử dụng ordinal encoding
# Áp dụng Ordinal Encoding (Xử lý biến thứ bậc)
ordinal_mappings = {
    "age_band": {"0-35": 0.0, "35-55": 0.5, "55<=": 1.0},
    "gender": {"M": 0.0, "F": 1.0},
    "highest_education": {
        "No Formal quals": 0.0,
        "Lower Than A Level": 0.25,
        "A Level or Equivalent": 0.5,
        "HE Qualification": 0.75,
        "Post Graduate Qualification": 1.0,
    },
    "imd_band": {
        "0-10%": 0.0, "10-20%": 0.1, "20-30%": 0.2, "30-40%": 0.3,
        "40-50%": 0.4, "50-60%": 0.5, "60-70%": 0.6, "70-80%": 0.7,
        "80-90%": 0.8, "90-100%": 0.9, "?": 0.5
    },
    "disability": {"N": 0.0, "Y": 1.0},
}

# Tạo bản sao để xử lý
df_encoded = df_merged.copy()

# Duyệt qua dictionary và map dữ liệu
for col, mapping in ordinal_mappings.items():
    if col in df_encoded.columns:
        df_encoded[col] = df_encoded[col].map(mapping)
        # Fill những giá trị không map được nếu có bằng 0.5 hoặc giá trị mặc định
        df_encoded[col] = df_encoded[col].fillna(0.5)

# Áp dụng One-Hot Encoding cho các biến static còn lại
# Lúc này age_band, gender... đã thành số (float), nên select_dtypes sẽ tự bỏ qua chúng
# Chỉ còn lại các cột như 'region', 'disability' v.v.
# remaining_cat_cols = df_encoded[static_cols].select_dtypes(include=['object']).columns

# print(f"Các cột sẽ One-Hot Encoding: {remaining_cat_cols}")

# df_encoded = pd.get_dummies(df_encoded, columns=remaining_cat_cols, drop_first=True)

# 5.5 Cập nhật danh sách Static Columns cuối cùng
# Lấy tất cả feature trong df_encoded ngoại trừ feature bị loại bỏ và feature time-series
final_static_cols = [c for c in df_encoded.columns if c not in (exclude_cols + all_ts_cols + ['target'])]

# Print final count and list of static features for verification
print(f"------------------------------")
print(f"Số lượng Feature tĩnh cuối cùng: {len(final_static_cols)}")
print(f"Danh sách: {final_static_cols}")
print(f"------------------------------")


Các cột Static được sử dụng: ['gender', 'highest_education', 'imd_band', 'age_band', 'num_of_prev_attempts', 'studied_credits', 'disability']
------------------------------
Số lượng Feature tĩnh cuối cùng: 7
Danh sách: ['gender', 'highest_education', 'imd_band', 'age_band', 'num_of_prev_attempts', 'studied_credits', 'disability']
------------------------------


In [13]:
# 6. Normalization
X_static = df_encoded[final_static_cols].values
X_ts_click = df_encoded[ts_click_cols].values
X_ts_score = df_encoded[ts_score_cols].values
X_ts_late = df_encoded[ts_late_cols].values
X_ts_weight = df_encoded[ts_weight_cols].values
y = df_encoded['target'].values

# Chia tập Train/Test (phải split trước khi scale để tránh data leakage)
# Lưu ý: Ta chia index để lát nữa lấy lại đúng phần dữ liệu cho từng loại
indices = np.arange(len(y))
X_train_idx, X_test_idx, y_train, y_test = train_test_split(indices, y, test_size=0.2, random_state=42)

def scale_data(train_idx, test_idx, data):
    scaler = MinMaxScaler()
    # Fit trên train, transform trên cả train và test
    train_scaled = scaler.fit_transform(data[train_idx])
    test_scaled = scaler.transform(data[test_idx])
    return train_scaled, test_scaled

X_train_static, X_test_static = scale_data(X_train_idx, X_test_idx, X_static)
X_train_click, X_test_click   = scale_data(X_train_idx, X_test_idx, X_ts_click)
X_train_score, X_test_score   = scale_data(X_train_idx, X_test_idx, X_ts_score)
X_train_late, X_test_late     = scale_data(X_train_idx, X_test_idx, X_ts_late)
X_train_weight, X_test_weight = scale_data(X_train_idx, X_test_idx, X_ts_weight)

## Số lượng mẫu trước smote

In [14]:
# 1. Gộp tất cả lại thành một bảng lớn duy nhất
X_train_combined = np.hstack([
    X_train_static,
    X_train_click,
    X_train_score,
    X_train_late,
    X_train_weight
])

# 2. Chạy SMOTE trên bảng lớn đó
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train_combined, y_train)

# 3. Tách ngược lại (Slice) bằng cách cộng dồn vị trí
# Chúng ta ghi rõ con số ra để không bị rối
w_static = X_train_static.shape[1]
w_click  = X_train_click.shape[1]
w_score  = X_train_score.shape[1]
w_late   = X_train_late.shape[1]
w_weight = X_train_weight.shape[1]

# Cắt từ X_res (Cấu trúc: [Static][Click][Score][Late][Weight])
start = 0
X_train_static_smote = X_res[:, start : start + w_static]

start += w_static
X_train_click_smote  = X_res[:, start : start + w_click]

start += w_click
X_train_score_smote  = X_res[:, start : start + w_score]

start += w_score
X_train_late_smote   = X_res[:, start : start + w_late]

start += w_late
X_train_weight_smote = X_res[:, start : start + w_weight]

print(f"Kiểm tra: Tổng số cột ban đầu là {X_train_combined.shape[1]}, tổng sau tách là {X_train_static_smote.shape[1] + X_train_click_smote.shape[1] + X_train_score_smote.shape[1] + X_train_late_smote.shape[1] + X_train_weight_smote.shape[1]}")

Kiểm tra: Tổng số cột ban đầu là 167, tổng sau tách là 167


Các mô hình machine learning thường nhận đầu vào là 2D với hàng là mẫu và cột là các đặc trưng
nhưng mô hình LSTM nhận đầu vào là 3D theo chuẩn
(Samples, TimeSteps, Features)
với:
Samples: số lượng sinh viên
timesteps: độ dài chuỗi thời gian
features: tổng số thông tin tại một thời điểm cụ thể


In [15]:
# 7. TẠO MẢNG 3D CHO CNN-LSTM
# Input shape mong muốn: (Samples, TimeSteps, Features)
# Features = 3 (Click, Score, Late) + N (Static features được lặp lại)
def create_multi_channel_input(static, click, score, late, weight, n_timesteps=40):
    n_samples = static.shape[0]

    # 1. Reshape các kênh Time-series từ (Samples, 40) thành (Samples, 40, 1)
    click_reshaped = click.reshape(n_samples, n_timesteps, 1)
    score_reshaped = score.reshape(n_samples, n_timesteps, 1)
    late_reshaped  = late.reshape(n_samples, n_timesteps, 1)
    weight_reshaped = weight.reshape(n_samples, n_timesteps, 1)

    # 2. Xử lý dữ liệu tĩnh (Static): Lặp lại cho mỗi bước thời gian
    # (Samples, n_static) -> (Samples, 1, n_static) -> (Samples, 40, n_static)
    static_reshaped = static.reshape(n_samples, 1, -1)
    static_tiled = np.tile(static_reshaped, (1, n_timesteps, 1))

    # 3. Ghép tất cả theo trục Features (axis=2)
    # Kết quả sẽ có dạng: (Samples, 40, 4 + n_static)
    final_input = np.concatenate([
        click_reshaped,
        score_reshaped,
        late_reshaped,
        weight_reshaped,
        static_tiled
    ], axis=2)

    return final_input



In [19]:
# Gọi hàm với dữ liệu không smote
X_train_final = create_multi_channel_input(
    X_train_static,
    X_train_click,
    X_train_score,
    X_train_late,
    X_train_weight,
    n_timesteps=40
)

X_test_final = create_multi_channel_input(
    X_test_static,
    X_test_click,
    X_test_score,
    X_test_late,
    X_test_weight,
    n_timesteps=40
)

print("-" * 30)
print("DỮ LIỆU KHÔNG SMOTE ĐÃ SẴN SÀNG CHO MÔ HÌNH CNN-LSTM")
print(f"X_train_final shape: {X_train_final.shape}") # Sẽ là (Số mẫu, 40, Số features)
print(f"y_train_smote shape: {y_train.shape}")
print(f"X_test_final shape:  {X_test_final.shape}")
print("-" * 30)

------------------------------
DỮ LIỆU KHÔNG SMOTE ĐÃ SẴN SÀNG CHO MÔ HÌNH CNN-LSTM
X_train_final shape: (26074, 40, 11)
y_train_smote shape: (26074,)
X_test_final shape:  (6519, 40, 11)
------------------------------


In [18]:
# Tạo mảng với dữ liệu SMOTE

X_train_final_smote = create_multi_channel_input(
    X_train_static_smote,
    X_train_click_smote,
    X_train_score_smote,
    X_train_late_smote,
    X_train_weight_smote

)

# Tập Test giữ nguyên (không SMOTE)
X_test_final  = create_multi_channel_input(
    X_test_static,
    X_test_click,
    X_test_score,
    X_test_late,
    X_test_weight
)

# Cập nhật nhãn y_train thành nhãn đã SMOTE
y_train_final_smote = y_res

print("-" * 30)
print("DỮ LIỆU SMOTE ĐÃ SẴN SÀNG CHO CNN-LSTM")
print(f"X_train shape: {X_train_final_smote.shape}")
print(f"y_train shape: {y_train_final_smote.shape}")

------------------------------
DỮ LIỆU SMOTE ĐÃ SẴN SÀNG CHO CNN-LSTM
X_train shape: (27480, 40, 11)
y_train shape: (27480,)


## Model training

In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import classification_report


### helper function

In [22]:
def evaluate_model(model, X_test, y_test, class_names=['FAIL (Lớp 0)', 'PASS (Lớp 1)'], threshold=0.5):
    # 1. Dự đoán (Ra xác suất)
    y_pred_probs = model.predict(X_test)

    # 2. Chuyển về nhãn 0/1 (Ngưỡng)
    y_pred_labels = (y_pred_probs > threshold).astype(int)

    # 3. In báo cáo chi tiết
    print(classification_report(y_test, y_pred_labels, target_names=class_names))

    return y_pred_labels

### LSTM thuần

In [24]:
# Không smote
# 1. Cấu hình tham số
n_timesteps = X_train_final_smote.shape[1]
n_features = X_train_final_smote.shape[2]

model = Sequential()

# Layer 1: LSTM
# units=100: Số lượng tế bào nhớ
# return_sequences=False: Chỉ lấy output của bước thời gian cuối cùng (để phân loại)
# input_shape: Kích thước dữ liệu đầu vào
model.add(Input(shape=(n_timesteps, n_features)))
model.add(LSTM(units=100, return_sequences=False))

# Layer 2: Dropout
model.add(Dropout(0.5))

# Layer 3: Output (Dense)
# units=1 (binary classification)
# activation='sigmoid': Đưa ra xác suất từ 0 đến 1 ( >0.5 là Đậu, <=0.5 là Trượt)
model.add(Dense(units=1, activation='sigmoid'))

# 3. Compile mô hình
# Optimizer: Adam với learning rate 0.001
# Loss: 'binary_crossentropy'
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

# In tóm tắt mô hình
model.summary()

# 4. training
# epochs=10, batch_size=32
history = model.fit(
    X_train_final, y_train,
    validation_data=(X_test_final, y_test),
    epochs=10,
    batch_size=32,
    verbose=1
)

# Gọi hàm đánh giá sau khi huấn luyện xong
print("\n Báo cáo đánh giá mô hình")
y_pred_labels = evaluate_model(model, X_test_final, y_test)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 100)            │        44,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,901 (175.39 KB)

 Trainable params: 44,901 (175.39 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
815/815 ━━━━━━━━━━━━━━━━━━━━ 52s 59ms/step - accuracy: 0.8506 - loss: 0.3309 - precision_1: 0.8085 - recall_1: 0.9006 - val_accuracy: 0.9161 - val_loss: 0.2253 - val_precision_1: 0.8614 - val_recall_1: 0.9780
Epoch 2/10
815/815 ━━━━━━━━━━━━━━━━━━━━ 75s 51ms/step - accuracy: 0.9185 - loss: 0.2181 - precision_1: 0.8731 - recall_1: 0.9660 - val_accuracy: 0.9182 - val_loss: 0.2183 - val_precision_1: 0.8630 - val_recall_1: 0.9810
Epoch 3/10
815/815 ━━━━━━━━━━━━━━━━━━━━ 52s 63ms/step - accuracy: 0.9233 - loss: 0.2043 - precision_1: 0.8816 - recall_1: 0.9684 - val_accuracy: 0.9241 - val_loss: 0.2050 - val_precision_1: 0.8746 - val_recall_1: 0.9780
Epoch 4/10
815/815 ━━━━━━━━━━━━━━━━━━━━ 53s 65ms/step - accuracy: 0.9286 - loss: 0.1901 - precision_1: 0.8857 - recall_1: 0.9732 - val_accuracy: 0.9265 - val_loss: 0.1839 - val_precision_1: 0.8867 - val_recall_1: 0.9666
Epoch 5/10
815/815 ━━━━━━━━━━━━━━━━━━━━ 41s 50ms/step - accuracy: 0.9316 - loss: 0.1716 - precision_1: 0.8963 - recall_1

In [25]:
# Sử dụng dữ liệu SMOTE
# 1. Cấu hình tham số
n_timesteps = X_train_final_smote.shape[1]
n_features = X_train_final_smote.shape[2]

model = Sequential()

# Layer 1: LSTM
# units=100: Số lượng tế bào nhớ
# return_sequences=False: Chỉ lấy output của bước thời gian cuối cùng (để phân loại)
# input_shape: Kích thước dữ liệu đầu vào
model.add(Input(shape=(n_timesteps, n_features)))
model.add(LSTM(units=100, return_sequences=False))

# Layer 2: Dropout
model.add(Dropout(0.5))

# Layer 3: Output (Dense)
# units=1 (binary classification)
# activation='sigmoid': Đưa ra xác suất từ 0 đến 1 ( >0.5 là Đậu, <=0.5 là Trượt)
model.add(Dense(units=1, activation='sigmoid'))

# 3. Compile mô hình
# Optimizer: Adam với learning rate 0.001
# Loss: 'binary_crossentropy'
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

# In tóm tắt mô hình
model.summary()

# 4. training
# epochs=10, batch_size=32
history = model.fit(
    X_train_final_smote, y_train_final_smote,
    validation_data=(X_test_final, y_test),
    epochs=10,
    batch_size=32,
    verbose=1
)

# Gọi hàm đánh giá sau khi huấn luyện xong
print("\n Báo cáo đánh giá mô hình")
y_pred_labels = evaluate_model(model, X_test_final, y_test)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 100)            │        44,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,901 (175.39 KB)

 Trainable params: 44,901 (175.39 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 31s 33ms/step - accuracy: 0.8475 - loss: 0.3400 - precision_2: 0.8241 - recall_2: 0.8836 - val_accuracy: 0.9139 - val_loss: 0.2256 - val_precision_2: 0.8796 - val_recall_2: 0.9456
Epoch 2/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 40s 32ms/step - accuracy: 0.9249 - loss: 0.2099 - precision_2: 0.8887 - recall_2: 0.9703 - val_accuracy: 0.9179 - val_loss: 0.2144 - val_precision_2: 0.8764 - val_recall_2: 0.9600
Epoch 3/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 42s 33ms/step - accuracy: 0.9266 - loss: 0.2077 - precision_2: 0.8896 - recall_2: 0.9749 - val_accuracy: 0.9201 - val_loss: 0.2069 - val_precision_2: 0.8738 - val_recall_2: 0.9692
Epoch 4/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 41s 33ms/step - accuracy: 0.9320 - loss: 0.1892 - precision_2: 0.8965 - recall_2: 0.9760 - val_accuracy: 0.9247 - val_loss: 0.1885 - val_precision_2: 0.8853 - val_recall_2: 0.9639
Epoch 5/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 28s 33ms/step - accuracy: 0.9313 - loss: 0.1802 - precision_2: 0.8999 - recall_2

### Mô hình hybrid CNN-LSTM

In [30]:

# 1. THIẾT LẬP THAM SỐ (tham số được thiết lập theo bài báo gốc)
input_shape = (n_timesteps, n_features)  # (TimeSteps, Features)
n_filters = 64          # Số bộ lọc CNN
kernel_size = 3         # Kích thước cửa sổ trượt
pool_size = 2           # Kích thước giảm mẫu
lstm_units = 100        # Số tế bào nhớ LSTM
dropout_rate = 0.5      # Tỷ lệ Dropout
learning_rate = 0.001   # Tốc độ học tối ưu
n_epochs = 20           # Số vòng lặp
batch_size = 32         # Kích thước lô

# 2. Model
model = Sequential(name="Hybrid_CNN_LSTM")

# Layer 1: Input
model.add(Input(shape=input_shape))

# Layer 2: Convolutional Neural Network (CNN)
# Trích xuất các đặc trưng cục bộ từ chuỗi hành vi
model.add(Conv1D(filters=n_filters,
                 kernel_size=kernel_size,
                 activation='relu',
                 padding='same')) # Padding same để giữ độ dài chuỗi ổn định trước khi pool

# Layer 3: Max Pooling
# Giảm chiều dữ liệu, giữ lại những đặc trưng nổi bật nhất
model.add(MaxPooling1D(pool_size=pool_size))

# Layer 4: Long Short-Term Memory
# Học các phụ thuộc thời gian từ các đặc trưng đã được CNN lọc
# Sau Pooling, chiều thời gian giảm đi một nửa (40 -> 20), giúp LSTM chạy nhanh hơn
model.add(LSTM(units=lstm_units, return_sequences=False))

# Layer 5: Dropout
model.add(Dropout(dropout_rate))

# Layer 6: Output
# Binary Classification
model.add(Dense(units=1, activation='sigmoid'))

# 3. COMPILE MÔ HÌNH
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])

model.summary()

# 4. train
print(f"\nBắt đầu huấn luyện CNN-LSTM trong {n_epochs} epochs...")
history = model.fit(
    X_train_final, y_train,
    validation_data=(X_test_final, y_test),
    epochs=n_epochs,
    batch_size=batch_size,
    verbose=1
)


Model: "Hybrid_CNN_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 40, 64)         │         2,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 20, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 100)            │        66,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 68,277 (266.71 KB)

 Trainable params: 68,277 (266.71 KB)

 Non-trainable params: 0 (0.00 B)


Bắt đầu huấn luyện CNN-LSTM trong 20 epochs...
Epoch 1/20
815/815 ━━━━━━━━━━━━━━━━━━━━ 25s 24ms/step - accuracy: 0.8679 - loss: 0.3097 - precision: 0.8411 - recall: 0.8789 - val_accuracy: 0.9144 - val_loss: 0.2129 - val_precision: 0.8968 - val_recall: 0.9233
Epoch 2/20
815/815 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - accuracy: 0.9283 - loss: 0.1878 - precision: 0.8930 - recall: 0.9628 - val_accuracy: 0.9238 - val_loss: 0.1873 - val_precision: 0.8870 - val_recall: 0.9594
Epoch 3/20
815/815 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - accuracy: 0.9305 - loss: 0.1773 - precision: 0.8964 - recall: 0.9661 - val_accuracy: 0.9270 - val_loss: 0.1715 - val_precision: 0.8856 - val_recall: 0.9692
Epoch 4/20
815/815 ━━━━━━━━━━━━━━━━━━━━ 21s 24ms/step - accuracy: 0.9377 - loss: 0.1593 - precision: 0.9067 - recall: 0.9675 - val_accuracy: 0.9348 - val_loss: 0.1697 - val_precision: 0.9057 - val_recall: 0.9607
Epoch 5/20
815/815 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - accuracy: 0.9375 - loss: 0.1565 - precision: 0.9

In [33]:

# 1. THIẾT LẬP THAM SỐ (tham số được thiết lập theo bài báo gốc)
input_shape = (n_timesteps, n_features)  # (TimeSteps, Features)
n_filters = 64          # Số bộ lọc CNN
kernel_size = 3         # Kích thước cửa sổ trượt
pool_size = 2           # Kích thước giảm mẫu
lstm_units = 100        # Số tế bào nhớ LSTM
dropout_rate = 0.5      # Tỷ lệ Dropout
learning_rate = 0.001   # Tốc độ học tối ưu
n_epochs = 20           # Số vòng lặp
batch_size = 32         # Kích thước lô

# 2. Model
model = Sequential(name="Hybrid_CNN_LSTM")

# Layer 1: Input
model.add(Input(shape=input_shape))

# Layer 2: Convolutional Neural Network (CNN)
# Trích xuất các đặc trưng cục bộ từ chuỗi hành vi
model.add(Conv1D(filters=n_filters,
                 kernel_size=kernel_size,
                 activation='relu',
                 padding='same')) # Padding same để giữ độ dài chuỗi ổn định trước khi pool

# Layer 3: Max Pooling
# Giảm chiều dữ liệu, giữ lại những đặc trưng nổi bật nhất
model.add(MaxPooling1D(pool_size=pool_size))

# Layer 4: Long Short-Term Memory
# Học các phụ thuộc thời gian từ các đặc trưng đã được CNN lọc
# Sau Pooling, chiều thời gian giảm đi một nửa (40 -> 20), giúp LSTM chạy nhanh hơn
model.add(LSTM(units=lstm_units, return_sequences=False))

# Layer 5: Dropout
model.add(Dropout(dropout_rate))

# Layer 6: Output
# Binary Classification
model.add(Dense(units=1, activation='sigmoid'))

# 3. COMPILE MÔ HÌNH
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])

model.summary()

# 4. train
print(f"\nBắt đầu huấn luyện CNN-LSTM trong {n_epochs} epochs...")
history = model.fit(
    X_train_final_smote, y_train_final_smote,
    validation_data=(X_test_final, y_test),
    epochs=n_epochs,
    batch_size=batch_size,
    verbose=1
)


Model: "Hybrid_CNN_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 40, 64)         │         2,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 20, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 100)            │        66,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 68,277 (266.71 KB)

 Trainable params: 68,277 (266.71 KB)

 Non-trainable params: 0 (0.00 B)


Bắt đầu huấn luyện CNN-LSTM trong 20 epochs...
Epoch 1/20
859/859 ━━━━━━━━━━━━━━━━━━━━ 28s 29ms/step - accuracy: 0.8713 - loss: 0.2999 - precision: 0.8359 - recall: 0.9270 - val_accuracy: 0.9187 - val_loss: 0.2078 - val_precision: 0.8648 - val_recall: 0.9794
Epoch 2/20
859/859 ━━━━━━━━━━━━━━━━━━━━ 21s 24ms/step - accuracy: 0.9257 - loss: 0.1965 - precision: 0.8913 - recall: 0.9699 - val_accuracy: 0.9271 - val_loss: 0.1845 - val_precision: 0.8854 - val_recall: 0.9698
Epoch 3/20
859/859 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - accuracy: 0.9303 - loss: 0.1829 - precision: 0.9001 - recall: 0.9701 - val_accuracy: 0.9297 - val_loss: 0.1739 - val_precision: 0.8902 - val_recall: 0.9695
Epoch 4/20
859/859 ━━━━━━━━━━━━━━━━━━━━ 22s 25ms/step - accuracy: 0.9386 - loss: 0.1610 - precision: 0.9121 - recall: 0.9702 - val_accuracy: 0.9343 - val_loss: 0.1602 - val_precision: 0.8895 - val_recall: 0.9816
Epoch 5/20
859/859 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - accuracy: 0.9428 - loss: 0.1526 - precision: 0.9

## tìm tham số tốt nhất

In [ ]:
import keras_tuner as kt

def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(40, 22))) # Input của bạn

    # Tuner cho số filters CNN
    hp_filters = hp.Int('filters', min_value=32, max_value=128, step=32)
    model.add(Conv1D(filters=hp_filters, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2))

    # Tuner cho số units LSTM
    hp_lstm_units = hp.Int('lstm_units', min_value=50, max_value=200, step=50)
    model.add(LSTM(units=hp_lstm_units, return_sequences=False))

    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Tuner cho Learning Rate
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Thiết lập Random Search giống bài báo
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10, # Thử 10 cấu hình khác nhau
    executions_per_trial=1,
    directory='my_dir',
    project_name='cnn_lstm_tuning'
)

# Bắt đầu tìm kiếm
tuner.search(X_train_final, y_train_res, epochs=10, validation_data=(X_test_final, y_test))

# Lấy tham số tốt nhất
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
Tham số tối ưu tìm được:
- Filters: {best_hps.get('filters')}
- LSTM Units: {best_hps.get('lstm_units')}
- Learning Rate: {best_hps.get('learning_rate')}
""")

Trial 10 Complete [00h 03m 48s]
val_accuracy: 0.9265224933624268

Best val_accuracy So Far: 0.929283618927002
Total elapsed time: 01h 01m 38s

Tham số tối ưu tìm được:
- Filters: 32
- LSTM Units: 150
- Learning Rate: 0.01



## Train mô hình với các khoảng thời gian
"Train model phù hợp với những lát cắt thời gian: (code)
- Mốc sớm tuần thứ 6. Độ chính xác kỳ vọng: 70 - 75%
- Mốc vàng tuần thứ 14: khoảng 1/3 thời lượng khóa học. Độ chính xác kì vọng: 85% - 90%
- Mốc chốt tuần thứ 20: khoảng 50% thời lượng. Độ chính xác kì vọng >95%
Note:
- Với mỗi mốc sẽ train để có bộ tham số phù hợp, theo dõi kết quả dự đoán
 (train lại thì cần nhưng không bắt buộc tuning tham số lại)"

In [ ]:
# 1. HÀM CẮT DỮ LIỆU TRỰC TIẾP TỪ 3D ARRAY
def slice_3d_data(X_3d, cutoff):
    return X_3d[:, :cutoff, :]

# 2. HÀM XÂY DỰNG MODEL LINH HOẠT
def build_model_for_cutoff(cutoff, n_features):
    model = Sequential(name=f"CNN_LSTM_Week_{cutoff}")
    model.add(Input(shape=(cutoff, n_features)))

    # TINH CHỈNH KIẾN TRÚC THEO ĐỘ DÀI CHUỖI
    if cutoff <= 8:
        model.add(Conv1D(filters=32, kernel_size=2, activation='relu', padding='same'))
        # Không dùng Pooling cho chuỗi siêu ngắn
    else:
        model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
        model.add(MaxPooling1D(pool_size=2))

    model.add(LSTM(units=100, return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# 3. CHẠY THỰC NGHIỆM (MAIN LOOP)
checkpoints = [6, 14, 20]
results = {}

# Đảm bảo bạn đang dùng biến X_train_final (đã SMOTE) và X_test_final
for week in checkpoints:
    print(f"\n>>> ĐANG HUẤN LUYỆN MỐC: TUẦN {week} <<<")

    # 1. Cắt dữ liệu
    X_tr_cut = slice_3d_data(X_train_final_smote, week)
    X_te_cut = slice_3d_data(X_test_final, week)

    print(f"Input Shape: {X_tr_cut.shape}")

    # 2. Build model
    model = build_model_for_cutoff(week, n_features=X_tr_cut.shape[2])

    # 3. Train
    # Với tuần 6, ta train ít epochs hơn chút để tránh học vẹt trên dữ liệu nhiễu
    epochs = 10 if week <= 8 else 15

    model.fit(X_tr_cut, y_train_final_smote,
              validation_data=(X_te_cut, y_test),
              epochs=epochs,
              batch_size=32,
              verbose=0) # verbose=0 cho gọn, hoặc để 1 nếu muốn xem

    # 4. Đánh giá
    y_pred = (model.predict(X_te_cut) > 0.7).astype(int)
    print(f"--- KẾT QUẢ TUẦN {week} ---")
    print(classification_report(y_test, y_pred, target_names=['FAIL', 'PASS']))

    # Lưu Accuracy
    acc = np.mean(y_pred.flatten() == y_test)
    results[f'Week_{week}'] = acc

print("\nTỔNG KẾT ACCURACY:")
for w, acc in results.items():
    print(f"{w}: {acc:.2%}")

### Kiểm tra ngưỡng để ưu tiên recall của lớp fail cao

In [ ]:
from sklearn.metrics import recall_score

In [ ]:
import os
from tensorflow.keras.models import load_model

# ==========================================
# CẤU HÌNH ĐƯỜNG DẪN LƯU MODEL
# ==========================================
# Tạo thư mục để lưu các model cho gọn
import os
if not os.path.exists('saved_models'):
    os.makedirs('saved_models')

checkpoints = [6, 14, 20]
thresholds_to_try = [0.5, 0.6, 0.7, 0.75]

for week in checkpoints:
    print("\n" + "="*60)
    print(f">>> ĐANG XỬ LÝ MỐC: TUẦN {week} <<<")

    # Tên file model sẽ lưu
    model_filename = f"saved_models/cnn_lstm_week_{week}.keras"

    # 1. Cắt dữ liệu (Luôn cần làm để có dữ liệu test)
    X_tr_cut = slice_3d_data(X_train_final_smote, week)
    X_te_cut = slice_3d_data(X_test_final, week)

    # ======================================================
    # 2. KIỂM TRA: LOAD HAY TRAIN MỚI? (PHẦN QUAN TRỌNG NHẤT)
    # ======================================================
    if os.path.exists(model_filename):
        print(f"✅ Đã tìm thấy file '{model_filename}'. Đang load model...")
        model = load_model(model_filename)
    else:
        print(f"⚠️ Chưa có file model. Bắt đầu huấn luyện mới...")

        # Build & Train
        model = build_model_for_cutoff(week, n_features=X_tr_cut.shape[2])
        epochs = 10 if week <= 8 else 15
        model.fit(X_tr_cut, y_train_final_smote,
                  validation_data=(X_te_cut, y_test),
                  epochs=epochs,
                  batch_size=32,
                  verbose=1) # Bật verbose lên để xem tiến độ train

        # LƯU MODEL NGAY SAU KHI TRAIN
        model.save(model_filename)
        print(f"💾 Đã lưu model vào '{model_filename}'")

    # ======================================================
    # 3. TINH CHỈNH NGƯỠNG (NHANH TỨC THÌ)
    # ======================================================
    print(f"--- Đang phân tích ngưỡng cho Tuần {week}... ---")

    # Dự đoán xác suất (Chỉ tốn vài giây)
    y_pred_prob = model.predict(X_te_cut, verbose=0)

    best_threshold = 0.5
    best_fail_recall = 0.0

    for t in thresholds_to_try:
        y_pred_t = (y_pred_prob > t).astype(int)
        rec_fail = recall_score(y_test, y_pred_t, pos_label=0)

        # Chỉ in ra nếu kết quả thú vị (để đỡ rối mắt)
        if t == 0.5 or t == 0.7:
            print(f"   [Ngưỡng {t}] -> Recall Fail: {rec_fail:.2%}")

        if rec_fail > best_fail_recall:
            best_fail_recall = rec_fail
            best_threshold = t

    print(f"👉 Khuyên dùng: Ngưỡng {best_threshold} (Recall Fail: {best_fail_recall:.2%})")


>>> ĐANG XỬ LÝ MỐC: TUẦN 6 <<<
✅ Đã tìm thấy file 'saved_models/cnn_lstm_week_6.keras'. Đang load model...
--- Đang phân tích ngưỡng cho Tuần 6... ---
   [Ngưỡng 0.5] -> Recall Fail: 73.59%
   [Ngưỡng 0.7] -> Recall Fail: 91.06%
👉 Khuyên dùng: Ngưỡng 0.75 (Recall Fail: 94.75%)

>>> ĐANG XỬ LÝ MỐC: TUẦN 14 <<<
⚠️ Chưa có file model. Bắt đầu huấn luyện mới...
Epoch 1/15
859/859 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.7210 - loss: 0.5251 - val_accuracy: 0.7888 - val_loss: 0.4278
Epoch 2/15
859/859 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.8088 - loss: 0.4156 - val_accuracy: 0.8007 - val_loss: 0.4058
Epoch 3/15
859/859 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.8221 - loss: 0.3890 - val_accuracy: 0.8168 - val_loss: 0.3907
Epoch 4/15
859/859 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.8232 - loss: 0.3820 - val_accuracy: 0.8116 - val_loss: 0.3916
Epoch 5/15
859/859 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.8247 - loss: 0.3850 - val_accuracy: 0.8222 - val_los